# Dynamic damages

In [116]:
import numpy as np
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()) + '/source')
import pickle
from utilities import dLambda
from supportfunctions import finiteDiff
import matplotlib.pyplot as plt
from numba import njit

In [115]:
solu = pickle.load(open('../data/solution/solu_dynamicdmg_51*101_0302_22:03', 'rb'))

In [129]:
solu['y_grid'][20]
numy_bar = solu['numy_bar']

In [135]:
solu['y_grid'][numy_bar-1]

2.0

In [136]:
# backward induction
rho = .9
mu2 = 1.86/1000
sigma2 = solu['sigma2']

In [203]:
solu_dict = solu['solu_dynamicdmg']
z_grid = solu['z_grid']
y_grid = solu['y_grid']
hz = solu['hz']
hy = solu['hy']
delta = .01
eta = .032
v_n = solu['v_n']

In [204]:
gamma2pList = np.array([0, 2*0.0197])
v0 = solu_dict[0]['phi'][:, :numy_bar]
v1 = solu_dict[gamma2pList[0]]['phi'][:, numy_bar-1]
v2 = solu_dict[gamma2pList[1]]['phi'][:, numy_bar-1]

In [205]:
numdmg = 3
dmg_weights = [.5, .5]
terminal_condition = np.average([v1, v2], axis=0, weights=dmg_weights)

In [206]:
numz = solu["numz"]
numy = 50
y_min = 0
y_max = 2
y_grid_new = np.linspace(y_min, y_max, numy)

(z_mat, y_mat) = np.meshgrid(z_grid, y_grid_new, indexing='ij')
stateSpace = np.hstack([z_mat.reshape(-1,1,order='F'), y_mat.reshape(-1,1,order='F')])

In [207]:
gamma2pMat = np.zeros((numdmg-1, numz, numy))
gamma2pMat[0] = gamma2pList[0]
gamma2pMat[1] = gamma2pList[1]
dlambdaMat = dLambda(y_mat, 1, 0.00017675, 2*0.0022, np.sum(gamma2pList*dmg_weights, axis=0), 2)

In [208]:
y_mat[0]

array([0.        , 0.04081633, 0.08163265, 0.12244898, 0.16326531,
       0.20408163, 0.24489796, 0.28571429, 0.32653061, 0.36734694,
       0.40816327, 0.44897959, 0.48979592, 0.53061224, 0.57142857,
       0.6122449 , 0.65306122, 0.69387755, 0.73469388, 0.7755102 ,
       0.81632653, 0.85714286, 0.89795918, 0.93877551, 0.97959184,
       1.02040816, 1.06122449, 1.10204082, 1.14285714, 1.18367347,
       1.2244898 , 1.26530612, 1.30612245, 1.34693878, 1.3877551 ,
       1.42857143, 1.46938776, 1.51020408, 1.55102041, 1.59183673,
       1.63265306, 1.67346939, 1.71428571, 1.75510204, 1.79591837,
       1.83673469, 1.87755102, 1.91836735, 1.95918367, 2.        ])

In [221]:
def pde_solver(v0, z_mat, y_mat, terminal_condition, 
               dmg_params, model_params, epsilon=.3,):
    
    gamma_1, gamma_2, gamma2pList, gamma_bar, dmg_weights = dmg_params
    delta, eta, mu2, sigma2, rho = model_params
    numz, numy = z_mat.shape
    hz = z_mat[1,0] - z_mat[0,0]
    hy = y_mat[0,1] - y_mat[0,0]
    v0 = v0.reshape(z_mat.shape, order='F')
    v0_dy = finiteDiff(v0, 1, 1, hy)
    dlambdaMat = dLambda(y_mat, 1, gamma_1, gamma_2, np.sum(gamma2pList*dmg_weights, axis=0), gamma_bar)
    ems = -delta*eta/(v0_dy*z_mat + v_n*dlambdaMat*z_mat)
    ems[ems<=0] = 1e-16
    # coefficient matrix
    LHS = np.zeros((numy*numz, numy*numz))
    RHS = np.zeros(numy*numz)
    for i in range(numz):
        for j in range(numy):
            idx = i*numy + j
            idx_yp1 = idx + 1
            idx_ym1 = idx - 1
            idx_zp1 = (i+1)*numy + j
            idx_zp2 = (i+2)*numy + j
            idx_zm1 = (i-1)*numy + j
            idx_zm2 = (i-2)*numy + j
            phi = v0[i,j]
            z = z_mat[i,0]
            condition = terminal_condition[i]
            dlambda= dlambdaMat[i,j]
            temp_1 = -rho*(z-mu2)
            var = z*sigma2**2/2
            LHS[idx, idx] += - delta - 1/epsilon
            # assign coefficient
            # z grid relevant
            if i == 0:
                LHS[idx, idx] += -temp_1/hz + var/(hz**2)  
                LHS[idx, idx_zp1] += temp_1/hz - var*2/(hz**2)
                LHS[idx, idx_zp2] += var/(hz**2)
            elif i == numz-1:
                LHS[idx, idx] += temp_1/hz + var/(hz**2)
                LHS[idx, idx_zm1] += -temp_1/hz - var*2/(hz**2)
                LHS[idx, idx_zm2] += var/(hz**2)            
            else:
                LHS[idx, idx_zp1] += temp_1 * (1/hz) * (temp_1 > 0) + var/(hz**2)
                LHS[idx, idx] += temp_1*((-1/hz) * (temp_1 > 0) + (1/hz)*(temp_1 <= 0))- var*2/(hz**2) 
                LHS[idx, idx_zm1] +=  temp_1*(-1/hz)*(temp_1<=0)+ var/(hz**2)
            # y grid relevant
            if j == 0:
                LHS[idx, idx] +=  - z*ems[i,j]/hy
                LHS[idx, idx_yp1] += z*ems[i,j]/hy
                RHS[idx] = -1/epsilon*phi - delta*eta*np.log(ems[i,j]) - (eta-1)*dlambda*z*ems[i,j]
            elif j >= 1 and j < numy-1:
                LHS[idx, idx_yp1] += z*ems[i,j]*(1/hy)*(z*ems[i,j] > 0)
                LHS[idx, idx] += z*ems[i,j]*((-1/hy)*(z*ems[i,j] > 0) + (1/hy)*(z*ems[i,j] <= 0))
                LHS[idx, idx_ym1] += z*ems[i,j]* (-1/hy) * (z*ems[i,j] <= 0)
                RHS[idx] = -1/epsilon*phi - delta*eta*np.log(ems[i,j]) - (eta-1)*dlambda*z*ems[i,j]     
            elif j == numy-1:
                LHS[idx] = 0
                LHS[idx, idx] =   1
                RHS[idx] = condition
                
            else:
                raise Exception("error.")
#     phi_grid = np.linalg.solve(LHS, RHS)
    return LHS, RHS# phi_grid

In [222]:
def false_transient(phi_grid, z_mat, y_mat, terminal_condition,
                    dmg_params, model_params,
                    epsilon=.3, max_iter=10_000, tol=1e-8):
    error = 1
    episode = 0
    while error > tol and episode < max_iter:
        phi_grid_old = phi_grid.copy()
        LHS, RHS = pde_solver(phi_grid_old,  z_mat, y_mat, 
                              terminal_condition, dmg_params, model_params, epsilon)
        phi_grid, exit_code = bicg(csc_matrix(LHS), RHS)
        error = np.max(np.abs((phi_grid_old-phi_grid)/epsilon))
        episode += 1
        print('Episode: {}\t error: {}'.format(episode,error))
    return phi_grid

In [223]:
gamma_1 = solu['gamma_1']
gamma_2 = solu['gamma_2']
gamma_bar = solu['gamma_bar']

dmgParams = (gamma_1, gamma_2, gamma2pList, gamma_bar, dmg_weights)
modelParams = (delta, eta, mu2, sigma2, rho)

In [ ]:
phi = false_transient(np.zeros(len(stateSpace)), z_mat, y_mat, terminal_condition, dmgParams, modelParams)

Episode: 1	 error: 0.1090947123904706
Episode: 2	 error: 0.018763251952657812
Episode: 3	 error: 0.010697702957430007
Episode: 4	 error: 0.029989120198988042
Episode: 5	 error: 0.02819377032027021
Episode: 6	 error: 0.06827605919990758
Episode: 7	 error: 0.08585525177738425
Episode: 8	 error: 0.09986268813038698
Episode: 9	 error: 0.06393606733232556
Episode: 10	 error: 0.05091176789894989
Episode: 11	 error: 0.0919781878893809
Episode: 12	 error: 0.09500052034460099
Episode: 13	 error: 0.024103604888050703
Episode: 14	 error: 0.41017544520465243
Episode: 15	 error: 0.41363162422217625
Episode: 16	 error: 0.04296056145125283
Episode: 17	 error: 0.02864689938800074
Episode: 18	 error: 0.09742246281888632
Episode: 19	 error: 0.11472294753007806
Episode: 20	 error: 0.13626243699820123
Episode: 21	 error: 0.15085309814332754
Episode: 22	 error: 0.02772266523129903
Episode: 23	 error: 0.07136452488464627
Episode: 24	 error: 0.0432558382835723
Episode: 25	 error: 0.0526219106216532
Episode: 

Episode: 203	 error: 0.057520185583731905
Episode: 204	 error: 0.09822891424992214
Episode: 205	 error: 0.06533726001876852
Episode: 206	 error: 0.052047098571743784
Episode: 207	 error: 0.02620363657636524
Episode: 208	 error: 0.036968277410278444
Episode: 209	 error: 0.05382831370807054
Episode: 210	 error: 0.05671716961633778
Episode: 211	 error: 0.129824716702471
Episode: 212	 error: 0.811926474311606
Episode: 213	 error: 0.7713964576226089
Episode: 214	 error: 0.08252452391832858
Episode: 215	 error: 0.4340549532602406
Episode: 216	 error: 0.39888218678687476
Episode: 217	 error: 0.09075411332862686
Episode: 218	 error: 0.08284120947874035
Episode: 219	 error: 0.05698055359382395
Episode: 220	 error: 0.15311342000388678
Episode: 221	 error: 0.11296716604698667
Episode: 222	 error: 0.13787349200876664
Episode: 223	 error: 0.12677544790865544
Episode: 224	 error: 0.07077611684003045
Episode: 225	 error: 0.020421559029100065
Episode: 226	 error: 0.07986297381382317
Episode: 227	 erro

Episode: 405	 error: 0.09562141958590205
Episode: 406	 error: 0.04193949852751923
Episode: 407	 error: 0.013330885550856635
Episode: 408	 error: 0.04742168522810233
Episode: 409	 error: 0.06671892251599541
Episode: 410	 error: 0.04862770412580255
Episode: 411	 error: 0.045964679505496715
Episode: 412	 error: 0.04553597584156273
Episode: 413	 error: 0.08273285868688993
Episode: 414	 error: 0.12731215332425203
Episode: 415	 error: 0.12552351273219858
Episode: 416	 error: 0.06708208715072146
Episode: 417	 error: 0.11112579770251312
Episode: 418	 error: 0.09602544784367807
Episode: 419	 error: 0.05063596395069185
Episode: 420	 error: 0.02207451495710515
Episode: 421	 error: 0.07230242879316498
Episode: 422	 error: 0.044038788260214554
Episode: 423	 error: 0.06055277534782769
Episode: 424	 error: 0.07760361902311076
Episode: 425	 error: 0.1111436852987015
Episode: 426	 error: 0.255512874074522
Episode: 427	 error: 0.2573905546677768
Episode: 428	 error: 0.7564248235819864
Episode: 429	 erro

Episode: 607	 error: 0.06469999430891553
Episode: 608	 error: 0.05570254504452049
Episode: 609	 error: 0.07464065913093852
Episode: 610	 error: 0.05858095844502732
Episode: 611	 error: 0.04045161829316841
Episode: 612	 error: 0.026863637905266115
Episode: 613	 error: 0.11352678987576768
Episode: 614	 error: 0.10653761751472736
Episode: 615	 error: 0.04631207437436827
Episode: 616	 error: 0.0372386456144258
Episode: 617	 error: 0.03678361358561603
Episode: 618	 error: 0.049015796523751966
Episode: 619	 error: 0.04829715264089279
Episode: 620	 error: 0.04999697877262152
Episode: 621	 error: 0.03623701172445285
Episode: 622	 error: 0.031304511050907945
Episode: 623	 error: 0.06393298260225061
Episode: 624	 error: 0.05016987173655172
Episode: 625	 error: 0.09668686719941241
Episode: 626	 error: 0.09247310567183994
Episode: 627	 error: 0.059226036212125174
Episode: 628	 error: 0.02505480038919962
Episode: 629	 error: 0.05142673460003327
Episode: 630	 error: 0.1910501129468582
Episode: 631	 

Episode: 807	 error: 0.06890796129598536
Episode: 808	 error: 0.042588978565682334
Episode: 809	 error: 0.0364439213179393
Episode: 810	 error: 0.9118796653650845
Episode: 811	 error: 0.8922420544379488
Episode: 812	 error: 0.18592794471667207
Episode: 813	 error: 0.18568047786218425
Episode: 814	 error: 0.037828239812821296
Episode: 815	 error: 0.04132435257572315
Episode: 816	 error: 0.17521573774438623
Episode: 817	 error: 0.17616732805902827
Episode: 818	 error: 0.026979988662007612
Episode: 819	 error: 0.02215469857417785
Episode: 820	 error: 0.023971774978913167
Episode: 821	 error: 0.0984703340892844
Episode: 822	 error: 0.07095634131421862
Episode: 823	 error: 0.10427304141098939
Episode: 824	 error: 0.10353925216048847
Episode: 825	 error: 0.03306124193329441
Episode: 826	 error: 0.06953979269825313
Episode: 827	 error: 0.06825679187954008
Episode: 828	 error: 0.04578677758146893
Episode: 829	 error: 0.03095795553404385
Episode: 830	 error: 0.03009183194226503
Episode: 831	 er

...

Episode: 2575	 error: 0.2290245170963759
Episode: 2576	 error: 0.09067739516546383
Episode: 2577	 error: 0.09657242470181657
Episode: 2578	 error: 0.03214104422782837
Episode: 2579	 error: 0.06065943854267654
Episode: 2580	 error: 0.055384189747611896
Episode: 2581	 error: 0.05795528267317842
Episode: 2582	 error: 0.0787768510740691
Episode: 2583	 error: 0.08799400150147733
Episode: 2584	 error: 0.060379057231831686
Episode: 2585	 error: 0.03139212903531243
Episode: 2586	 error: 0.05770586269178815
Episode: 2587	 error: 0.08394558321446492
Episode: 2588	 error: 0.07523822264639399
Episode: 2589	 error: 0.015962547368833676
Episode: 2590	 error: 0.03299302783044342
Episode: 2591	 error: 0.23330881807337792
Episode: 2592	 error: 0.23113549369411984
Episode: 2593	 error: 0.03550393433915006
Episode: 2594	 error: 0.062382974690007795
Episode: 2595	 error: 0.11861319533094451
Episode: 2596	 error: 0.11816525484826398
Episode: 2597	 error: 0.03296541499986968
Episode: 2598	 error: 0.06231891

Episode: 2771	 error: 0.10226837250009771
Episode: 2772	 error: 0.07996444230258472
Episode: 2773	 error: 0.017593030215059514
Episode: 2774	 error: 0.025934478921037174
Episode: 2775	 error: 0.10487196456785489
Episode: 2776	 error: 0.08997647780076039
Episode: 2777	 error: 0.02018796643625219
Episode: 2778	 error: 0.02070870270098969
Episode: 2779	 error: 0.023280198265491422
Episode: 2780	 error: 0.023691904171281326
Episode: 2781	 error: 0.13679878853512573
Episode: 2782	 error: 0.14511821154777255
Episode: 2783	 error: 0.17295084779368028
Episode: 2784	 error: 0.1292091552214871
Episode: 2785	 error: 0.05391272769910006
Episode: 2786	 error: 0.03914805549640728
Episode: 2787	 error: 0.02871368536119803
Episode: 2788	 error: 0.05731149521738077
Episode: 2789	 error: 0.04837105813913402
Episode: 2790	 error: 0.08948192554141975
Episode: 2791	 error: 0.07758140702468194
Episode: 2792	 error: 0.03443833331051216
Episode: 2793	 error: 0.059220187815339165
Episode: 2794	 error: 0.053054

Episode: 2967	 error: 0.036493373646659874
Episode: 2968	 error: 0.17684022750311065
Episode: 2969	 error: 0.17498311848672743
Episode: 2970	 error: 0.11958327381968999
Episode: 2971	 error: 0.021860541083819428
Episode: 2972	 error: 0.04311622372673521
Episode: 2973	 error: 0.043524433975477235
Episode: 2974	 error: 0.04503097062006089
Episode: 2975	 error: 0.08294234742643589
Episode: 2976	 error: 0.1004086192580004
Episode: 2977	 error: 0.05594643942739015
Episode: 2978	 error: 0.05266157330275026
Episode: 2979	 error: 0.0619591133206347
Episode: 2980	 error: 0.031563730541299524
Episode: 2981	 error: 0.03158972291110491
Episode: 2982	 error: 0.06140768176728202
Episode: 2983	 error: 0.06138070500515114
Episode: 2984	 error: 0.07419490380323651
Episode: 2985	 error: 0.027743982049418647
Episode: 2986	 error: 0.07390207705909697
Episode: 2987	 error: 0.06874552854139883
Episode: 2988	 error: 0.17183578748555806
Episode: 2989	 error: 0.1565760034417033
Episode: 2990	 error: 0.02639604

Episode: 3163	 error: 0.12441782284977924
Episode: 3164	 error: 0.01884464395304066
Episode: 3165	 error: 0.036794504778708687
Episode: 3166	 error: 0.10966193448842747
Episode: 3167	 error: 0.12793515697603644
Episode: 3168	 error: 0.14503700543112966
Episode: 3169	 error: 0.13676708605609472
Episode: 3170	 error: 0.09111890598880489
Episode: 3171	 error: 0.04220073389705325
Episode: 3172	 error: 0.10989408565795472
Episode: 3173	 error: 0.1198653271377513
Episode: 3174	 error: 0.025511011219860984
Episode: 3175	 error: 0.03072819464990395
Episode: 3176	 error: 0.036867517721633755
Episode: 3177	 error: 0.03981483323718465
Episode: 3178	 error: 0.14785452537680374
Episode: 3179	 error: 0.14720346764010211
Episode: 3180	 error: 0.032236850075688515
Episode: 3181	 error: 0.12301055204448545
Episode: 3182	 error: 0.1309948936586133
Episode: 3183	 error: 0.02926433543944293
Episode: 3184	 error: 0.0337841634261528
Episode: 3185	 error: 0.17187059140871463
Episode: 3186	 error: 0.100328373

Episode: 3359	 error: 0.06745788618287346
Episode: 3360	 error: 0.08358817558452412
Episode: 3361	 error: 0.027418424033525242
Episode: 3362	 error: 0.04133632629120107
Episode: 3363	 error: 0.07864461893487765
Episode: 3364	 error: 0.06960160332034664
Episode: 3365	 error: 0.13673612984135686
Episode: 3366	 error: 0.0986546519198924
Episode: 3367	 error: 0.0812390470938247
Episode: 3368	 error: 0.062466203366146465
Episode: 3369	 error: 0.027903811392006525
Episode: 3370	 error: 0.13298574487436068
Episode: 3371	 error: 0.11120933862300254
Episode: 3372	 error: 0.07224113261365592
Episode: 3373	 error: 0.07076057339292707
Episode: 3374	 error: 0.03662406994322658
Episode: 3375	 error: 0.0390722017636436
Episode: 3376	 error: 0.03733125503945445
Episode: 3377	 error: 0.051089145477767973
Episode: 3378	 error: 0.04409551610967658
Episode: 3379	 error: 0.13316424015963604
Episode: 3380	 error: 0.13323782876990217
Episode: 3381	 error: 0.0391465124794254
Episode: 3382	 error: 0.0383164945

In [ ]:
phi = phi.reshape(V.shape)
dphi_dy = finiteDiff(phi,1,1,hy)

In [ ]:
ems = - delta*eta*(dphi_dy*z_mat + v_n*dlambdaMat*z_mat)

In [ ]:
plt.plot(phi[10])